# Preparation

<b>Load Libraries</b>

In [1]:
# data structure
import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
sns.set_theme('notebook')

<b>Load Dataset</b>

In [4]:
# dataset
df_base = pd.read_excel('../resource/data/raw.xlsx')